# Testing for E2 - using all spindles, training with all data
fistly define the UUID of the experiment to use

In [1]:
experimentId="6e8f728f-81ce-4048-8acf-f8d1ffc89a39"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [3]:
samplerate=200  #Should rethink this

In [4]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [5]:
#consider to include this in a function
minDuration=0.3
maxDuration=2.9
annotations=annotations[annotations.type=='spindle']
annotations=annotations[annotations.duration>minDuration]
annotations=annotations[annotations.duration<maxDuration]
annotations=annotations.reset_index(drop=True)

In [6]:
#check with EDA results (not needed)
print(1-len(annotations)/33458)

0.011297746428357902


In [7]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [8]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [9]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [10]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,9e4dd9bb-a7d9-4ee8-a762-d656d8308d91,0.018639
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,5e1bcb8a-f879-4f42-ae21-0e1ae715095f,0.020055
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,2ecef3ee-0879-42d7-bcf0-ecfa0e3e601a,0.021075
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,52b0d6a8-7f01-4694-85d4-14ef17a025b9,0.018331
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,32c9793a-2115-4e77-8b6f-e8f71d09cecb,0.019222
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,241d9889-61ff-47f7-97c2-f502ae3dba38,0.019625
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,f6a46037-1f32-40b5-a6ff-cd293b0e4203,0.017824
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,6cfc1f74-3653-47eb-a166-f59fefa13825,0.017542
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,68c5483d-9072-4478-8a87-a7af3324f690,0.015442
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,349fe35c-b749-431d-8e8c-c288b150e998,0.016460


In [11]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                          \
                        count      mean       std       min       25%   
criteriumName                                                           
E1                       15.0  0.018463  0.001834  0.015442  0.017419   
E2                       15.0  0.058566  0.004500  0.052646  0.055195   
union                    15.0  0.059793  0.004559  0.054024  0.056196   

                                             
                    50%       75%       max  
criteriumName                                
E1             0.018331  0.019424  0.022439  
E2             0.057005  0.061849  0.067263  
union          0.058468  0.062907  0.068772

In [12]:
featureSelection

,characteristic,bandName,window
21,hjortActivity,sigma,0.5
66,petrosian,broadband,0.5
22,hjortActivity,theta,0.5
79,relativePower,beta1,0.5
29,hjortComplexity,sigma,0.5


## Hyperparameter definition
this should come from a previous evaluation notebook

In [14]:
hyperClose=0.25
hyperDuration=0.3
hyperThres=0.4
hyperDepth=20

## Testing with E2 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [15]:
experimentModels=experimentModels[experimentModels.criteriumName=='E2'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,bf958847-77fc-4eab-9c95-7a30ef71f8ee,0.057005
1,0002,E2,[0002],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,f733d4b9-1730-40f9-91c2-f0ac2f36367e,0.061438
2,0002,E2,[0002],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,b7018fb5-07f6-4ca6-a2ae-9c1646bde5b9,0.063269
3,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,d47802dc-da95-4420-a6f5-a4bb291c9385,0.062012
4,0002,E2,[0002],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,556a03bf-82b3-4468-b1ff-c804988a238a,0.058957
5,0002,E2,[0002],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,d6976249-69b2-42de-a8b2-16adbd07e4b9,0.061686
6,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,3237920b-4e80-4a7b-83af-d66298816ba3,0.056088
7,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,92756d09-a80a-49f1-bbfc-f8a12570ddbe,0.052646
8,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,2d1def14-06c9-4a40-a577-752522c66ed5,0.055127
9,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,c930c4a8-32c1-41d8-a540-db8ec02d8c50,0.053302


In [16]:
#threshold for the metric by event
masterIoU=0.2

In [17]:
def getIou(coordA,coordB):
    if (coordA[1]<coordB[0])|(coordB[1]<coordA[0]): #NOT INTERSECTION
        iou=0
    else:   #INTERSECTION
        inter=np.min((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        union=np.max((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        iou=inter/union
    return iou

def annotationPairToMetrics(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #binarize
    binarized=iouMatrix>thresIoU
    #calculateF1
    outF1=(np.sum(np.max(binarized,axis=0))+np.sum(np.max(binarized,axis=1)))/(len(annotations)+len(detections))
    recall=np.sum(np.max(binarized,axis=1))/len(annotations)
    precision=np.sum(np.max(binarized,axis=0))/len(detections)
    return outF1,recall,precision

In [18]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [19]:
dumpPickle('experimentModelsTest_E2_temp.pkl',experimentModels)

In [20]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [21]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [22]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [23]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [24]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [25]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.593936 NaN  0.593936  0.593936  0.593936            1.0   
0002     1.0  0.828666 NaN  0.828666  0.828666  0.828666            1.0   
0003     1.0  0.737058 NaN  0.737058  0.737058  0.737058            1.0   
0005     1.0  0.719764 NaN  0.719764  0.719764  0.719764            1.0   
0006     1.0  0.677570 NaN  0.677570  0.677570  0.677570            1.0   
0007     1.0  0.653164 NaN  0.653164  0.653164  0.653164            1.0   
0009     1.0  0.765999 NaN  0.765999  0.765999  0.765999            1.0   
0010     1.0  0.773176 NaN  0.773176  0.773176  0.773176            1.0   
0011     1.0  0.759843 NaN  0.759843  0.759843  0.759843            1.0   
0012     1.0  0.675976 NaN  0.675976  0.675976  0.675976            1.0   
0013     1.0  0.667660 NaN  0.667660  0.667660  0.667660            1.0   
0014     1.0  0.582959 NaN  0.582959  0.582959  0.582959            1.0   
0017     1.0  0.774113 NaN  0.774113  0.774113  0.774113            1.0   
0018     1.0  0.710000 NaN  0.710000  0.710000  0.710000            1.0   
0019     1.0  0.690798 NaN  0.690798  0.690798  0.690798            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.863100 NaN  0.863100  0.863100  0.863100         1.0  0.452745 NaN   
0002  0.758919 NaN  0.758919  0.758919  0.758919         1.0  0.913361 NaN   
0003  0.723473 NaN  0.723473  0.723473  0.723473         1.0  0.751261 NaN   
0005  0.587794 NaN  0.587794  0.587794  0.587794         1.0  0.928149 NaN   
0006  0.751479 NaN  0.751479  0.751479  0.751479         1.0  0.616788 NaN   
0007  0.499334 NaN  0.499334  0.499334  0.499334         1.0  0.948243 NaN   
0009  0.832272 NaN  0.832272  0.832272  0.832272         1.0  0.708690 NaN   
0010  0.737482 NaN  0.737482  0.737482  0.737482         1.0  0.812923 NaN   
0011  0.630048 NaN  0.630048  0.630048  0.630048         1.0  0.958195 NaN   
0012  0.535374 NaN  0.535374  0.535374  0.535374         1.0  0.917962 NaN   
0013  0.507148 NaN  0.507148  0.507148  0.507148         1.0  0.979532 NaN   
0014  0.413518 NaN  0.413518  0.413518  0.413518         1.0  0.985461 NaN   
0017  0.765045 NaN  0.765045  0.765045  0.765045         1.0  0.783418 NaN   
0018  0.567528 NaN  0.567528  0.567528  0.567528         1.0  0.946763 NaN   
0019  0.534347 NaN  0.534347  0.534347  0.534347         1.0  0.975214 NaN   

                                    
           min       50%       max  
test                                
0001  0.452745  0.452745  0.452745  
0002  0.913361  0.913361  0.913361  
0003  0.751261  0.751261  0.751261  
0005  0.928149  0.928149  0.928149  
0006  0.616788  0.616788  0.616788  
0007  0.948243  0.948243  0.948243  
0009  0.708690  0.708690  0.708690  
0010  0.812923  0.812923  0.812923  
0011  0.958195  0.958195  0.958195  
0012  0.917962  0.917962  0.917962  
0013  0.979532  0.979532  0.979532  
0014  0.985461  0.985461  0.985461  
0017  0.783418  0.783418  0.783418  
0018  0.946763  0.946763  0.946763  
0019  0.975214  0.975214  0.975214

In [26]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.593936,0.863100,0.452745
1,0002,0.828666,0.758919,0.913361
2,0003,0.737058,0.723473,0.751261
3,0005,0.719764,0.587794,0.928149
4,0006,0.677570,0.751479,0.616788
5,0007,0.653164,0.499334,0.948243
6,0009,0.765999,0.832272,0.708690
7,0010,0.773176,0.737482,0.812923
8,0011,0.759843,0.630048,0.958195
9,0012,0.675976,0.535374,0.917962


In [27]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.073788e-01
eventPrecision    6.471241e-01
eventRecall       8.452471e-01
dtype: float64

In [28]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_12452\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.068525
eventPrecision    0.137210
eventRecall       0.156674
dtype: float64

In [29]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [30]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

## Restrict to only N2 stage

In [31]:
experimentModels_N2=experimentModels.copy()
experimentModels_N2.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate,rawF1,rawPrecision,rawRecall,f1,precision,recall,eventF1,eventPrecision,eventRecall
0,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,bf958847-77fc-4eab-9c95-7a30ef71f8ee,0.057005,0.379916,0.885529,0.241835,0.422104,0.886382,0.277009,0.593936,0.863100,0.452745
1,0002,E2,[0002],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,f733d4b9-1730-40f9-91c2-f0ac2f36367e,0.061438,0.705351,0.812784,0.623004,0.734703,0.794036,0.683620,0.828666,0.758919,0.913361
2,0002,E2,[0002],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,b7018fb5-07f6-4ca6-a2ae-9c1646bde5b9,0.063269,0.585866,0.751667,0.479991,0.627416,0.747121,0.540772,0.737058,0.723473,0.751261
3,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,d47802dc-da95-4420-a6f5-a4bb291c9385,0.062012,0.674378,0.642374,0.709738,0.681153,0.606250,0.777174,0.719764,0.587794,0.928149
4,0002,E2,[0002],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,556a03bf-82b3-4468-b1ff-c804988a238a,0.058957,0.514396,0.764121,0.387693,0.553065,0.756994,0.435692,0.677570,0.751479,0.616788


In [32]:
annotationsN2=pd.read_csv(datapath+'/MASS/annotations/spindlesFilteredN2.csv')
annotationsN2['samplerate']=samplerate
annotationsN2['subjectId']=annotationsN2.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
annotationsN2['labelerId']=annotationsN2.apply(
    lambda row: str(row.labelerId).zfill(4),axis=1)
annotationsN2['stopTime']=annotationsN2.apply(
    lambda row: row.startTime+row.duration , axis=1)
annotationsN2['startInd']=annotationsN2.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
annotationsN2['stopInd']=annotationsN2.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

In [33]:
## load hypnograms
stagesAnnotations=pd.read_csv(datapath+"\MASS\stages\stages.csv")
stagesAnnotations['samplerate']=samplerate
stagesAnnotations['subjectId']=stagesAnnotations.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
stagesAnnotations['stopTime']=stagesAnnotations.apply(
    lambda row: row.startTime+row.duration , axis=1)
stagesAnnotations['startInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
stagesAnnotations['stopInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)
stagesAnnotations.head(5)

hypnograms={}
for ind, row in signalsMetadata.iterrows():
    subjectId=row.subjectId
    thisStages=stagesAnnotations[stagesAnnotations.subjectId==subjectId]
    excerptDimension=int(row.duration*row.samplerate)
    thisHypnogram=np.ones((excerptDimension,))*np.nan
    for ind_stg, row_stg in thisStages.iterrows():
        thisHypnogram[row_stg.startInd:row_stg.stopInd]=row_stg.value
    hypnograms[subjectId]=thisHypnogram   

In [34]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels_N2.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels_N2)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    testHypnogram=hypnograms[testSubjectId]
    testN2Mask=testHypnogram==2
    #Define annotations criterium
    usedAnnotations=annotationsN2[annotationsN2.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres

    #Apply mask ---------------------------------->
    rawLabels=rawLabels*testN2Mask
    #<---------------------------------------------

    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)

    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels_N2['rawF1']=rawF1s
experimentModels_N2['rawPrecision']=rawPrecisions
experimentModels_N2['rawRecall']=rawRecalls

experimentModels_N2['f1']=f1s
experimentModels_N2['precision']=precisions
experimentModels_N2['recall']=recalls

experimentModels_N2['eventF1']=eventF1s
experimentModels_N2['eventPrecision']=eventPrecisions
experimentModels_N2['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [35]:
dumpPickle('experimentModelsTest_N2_E2_temp.pkl',experimentModels_N2)

In [36]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [37]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [38]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [39]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [40]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.620885 NaN  0.620885  0.620885  0.620885            1.0   
0002     1.0  0.857524 NaN  0.857524  0.857524  0.857524            1.0   
0003     1.0  0.815861 NaN  0.815861  0.815861  0.815861            1.0   
0005     1.0  0.793490 NaN  0.793490  0.793490  0.793490            1.0   
0006     1.0  0.723450 NaN  0.723450  0.723450  0.723450            1.0   
0007     1.0  0.780341 NaN  0.780341  0.780341  0.780341            1.0   
0009     1.0  0.788104 NaN  0.788104  0.788104  0.788104            1.0   
0010     1.0  0.868743 NaN  0.868743  0.868743  0.868743            1.0   
0011     1.0  0.810615 NaN  0.810615  0.810615  0.810615            1.0   
0012     1.0  0.804241 NaN  0.804241  0.804241  0.804241            1.0   
0013     1.0  0.754430 NaN  0.754430  0.754430  0.754430            1.0   
0014     1.0  0.652311 NaN  0.652311  0.652311  0.652311            1.0   
0017     1.0  0.837251 NaN  0.837251  0.837251  0.837251            1.0   
0018     1.0  0.759142 NaN  0.759142  0.759142  0.759142            1.0   
0019     1.0  0.694035 NaN  0.694035  0.694035  0.694035            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.983813 NaN  0.983813  0.983813  0.983813         1.0  0.453566 NaN   
0002  0.808848 NaN  0.808848  0.808848  0.808848         1.0  0.912981 NaN   
0003  0.890873 NaN  0.890873  0.890873  0.890873         1.0  0.752108 NaN   
0005  0.692114 NaN  0.692114  0.692114  0.692114         1.0  0.929661 NaN   
0006  0.871355 NaN  0.871355  0.871355  0.871355         1.0  0.618293 NaN   
0007  0.663415 NaN  0.663415  0.663415  0.663415         1.0  0.949359 NaN   
0009  0.884123 NaN  0.884123  0.884123  0.884123         1.0  0.709816 NaN   
0010  0.932858 NaN  0.932858  0.932858  0.932858         1.0  0.812337 NaN   
0011  0.702256 NaN  0.702256  0.702256  0.702256         1.0  0.959394 NaN   
0012  0.715535 NaN  0.715535  0.715535  0.715535         1.0  0.918544 NaN   
0013  0.614155 NaN  0.614155  0.614155  0.614155         1.0  0.979487 NaN   
0014  0.486962 NaN  0.486962  0.486962  0.486962         1.0  0.986049 NaN   
0017  0.896518 NaN  0.896518  0.896518  0.896518         1.0  0.785229 NaN   
0018  0.632994 NaN  0.632994  0.632994  0.632994         1.0  0.947177 NaN   
0019  0.538168 NaN  0.538168  0.538168  0.538168         1.0  0.975394 NaN   

                                    
           min       50%       max  
test                                
0001  0.453566  0.453566  0.453566  
0002  0.912981  0.912981  0.912981  
0003  0.752108  0.752108  0.752108  
0005  0.929661  0.929661  0.929661  
0006  0.618293  0.618293  0.618293  
0007  0.949359  0.949359  0.949359  
0009  0.709816  0.709816  0.709816  
0010  0.812337  0.812337  0.812337  
0011  0.959394  0.959394  0.959394  
0012  0.918544  0.918544  0.918544  
0013  0.979487  0.979487  0.979487  
0014  0.986049  0.986049  0.986049  
0017  0.785229  0.785229  0.785229  
0018  0.947177  0.947177  0.947177  
0019  0.975394  0.975394  0.975394

In [41]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.620885,0.983813,0.453566
1,0002,0.857524,0.808848,0.912981
2,0003,0.815861,0.890873,0.752108
3,0005,0.793490,0.692114,0.929661
4,0006,0.723450,0.871355,0.618293
5,0007,0.780341,0.663415,0.949359
6,0009,0.788104,0.884123,0.709816
7,0010,0.868743,0.932858,0.812337
8,0011,0.810615,0.702256,0.959394
9,0012,0.804241,0.715535,0.918544


In [42]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.706949e-01
eventPrecision    7.542658e-01
eventRecall       8.459597e-01
dtype: float64

In [43]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_12452\813125388.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.071824
eventPrecision    0.152561
eventRecall       0.156467
dtype: float64

In [44]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels_N2.eventPrecision,
    'event F1':experimentModels_N2.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels_N2.eventRecall,
    'event F1':experimentModels_N2.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [45]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### remove outliers

In [46]:
np.setdiff1d(experimentModels_N2.test,['0001','0014','0019'])

array(['0002', '0003', '0005', '0006', '0007', '0009', '0010', '0011',
       '0012', '0013', '0017', '0018'], dtype=object)

In [47]:
experimentModels_N2[experimentModels_N2.test.isin(np.setdiff1d(experimentModels_N2.test,['0001','0014','0019']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              1.666917e+43
eventF1           7.994327e-01
eventPrecision    7.754203e-01
eventRecall       8.561989e-01
dtype: float64

In [48]:
def annotationPairToGraph(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #create tables
    index0=np.apply_along_axis(np.argmax,0,iouMatrix)
    iou0=np.apply_along_axis(np.max,0,iouMatrix)
    index1=np.apply_along_axis(np.argmax,1,iouMatrix)
    iou1=np.apply_along_axis(np.max,1,iouMatrix)
    
    tableOut=pd.DataFrame({
        'indexGT':index0,
        'iou':iou0
    })

    tableGT=pd.DataFrame({
        'indexOut':index1,
        'iou':iou1
    })

    tableOut['type']='out'
    tableGT['type']='gt'
    tableOut['indexOut']=tableOut.index
    tableGT['indexGT']=tableGT.index
    #Correct external indexes of objects not overlapping
    tableOut.loc[tableOut.iou==0,'indexGT']='NA'
    tableGT.loc[tableGT.iou==0,'indexOut']='NA'
    #set tps
    tableOut['tp']=tableOut.iou>thresIoU
    tableOut['fp']=tableOut.iou<=thresIoU
    tableGT['tp']=tableGT.iou>thresIoU
    tableGT['fn']=tableGT.iou<=thresIoU
    #calculate metrics
    recall=np.sum(tableGT['tp'])/len(tableGT)
    precision=np.sum(tableOut['tp'])/len(tableOut)
    f1=(np.sum(tableGT['tp'])+np.sum(tableOut['tp']))/(len(tableGT)+len(tableOut))
    #concatenate tables
    appended=pd.concat(objs=(tableOut,tableGT),axis=0)
    #modify values
    appended['x']=appended['indexGT']
    appended['y']=appended['indexOut']
    appended.loc[((appended.type=='out')&(~ appended.tp)),'x']=-10
    appended.loc[((appended.type=='gt')&(~ appended.tp)),'y']=-10
    appended['size']=1
    appended.loc[appended.type=='out','size']=3
    #create the graph
    minTPIoU=np.min(appended[appended.tp].iou)
    fig=px.scatter(appended,x='x',y='y',color='iou',symbol='type',
    opacity=0.8,symbol_map={'out':'circle-open','gt':'circle'},size='size',
    color_continuous_scale=
        ((0.0, 'rgb(40,40,40)'),
        (0.000001, 'rgb(28,227,255)'),
        (0.14, 'rgb(56,199,255)'),
        (0.29, 'rgb(85,170,255)'),
        (0.42, 'rgb(113,142,255)'),
        (0.57, 'rgb(142,113,255)'),
        (0.71, 'rgb(170,85,255)'),
        (0.86, 'rgb(199,56,255)'),
        (1.0, 'rgb(227,28,255)')),
    range_x=(-20,len(tableGT)+10),range_y=(-20,len(tableOut)+10),
    title='by-Event evaluation summary<br><sup>F1(@IoU>'+str(thresIoU)+')='+str(round(f1,4))+' | minimum TP IoU: '+str(round(minTPIoU,4))+'</sup>',
    hover_data={'x':False,
    'y':False,
    'tp':False,
    'fp':False,
    'fn':False,
    'size':False,
    'type':False,
    'iou':':.4f', # customize hover for column of y attribute
    'indexGT':True,
    'indexOut':True
    })
    for t in fig.data:
        t.marker.line.width = 2
    fig.update_xaxes(title_text=str(len(tableGT))+' ANNOTATIONS | recall(@IoU>'+str(thresIoU)+')= '+str(round(recall,4)))
    fig.update_yaxes(title_text=str(len(tableOut))+' DETECTIONS | precision(@IoU>'+str(thresIoU)+')= '+str(round(precision,4)))
    fig.add_vline(x=-5,line_dash='dash')
    fig.add_hline(y=-5,line_dash='dash')
    #----------------------------------------------------------------------->
    # https://stackoverflow.com/questions/61827165/plotly-how-to-handle-overlapping-colorbar-and-legends
    # @vestland answer
    """ fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
                                            ticks="outside",
                                            ticksuffix=" bills")) """
    # @bitbang answer
    fig.update_layout(legend_orientation="h")
    #<----------------------------------------------------------------------
    return fig

In [49]:
annotationPairToGraph(gtAnnotations,processedAnnotations)